## Importing the Library

In [2]:
import cv2
import os
import numpy as np
from PIL import Image
from torchvision import transforms
from sklearn.model_selection import train_test_split
import imgaug.augmenters as iaa

## Data Collection

In [ ]:
def generate_dataset(label):
    
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    def augment_image(image):
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.RandomResizedCrop((160, 160), scale=(0.8, 1.0)),
        ])
        return transform(image)

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
         
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
     
    cap = cv2.VideoCapture(0)
    img_id = 0

    path_rgb = 'data/dataset_rgb/'
    label_rgb_path = os.path.join(path_rgb, label)
    os.makedirs(label_rgb_path, exist_ok= True)
     
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Covert to PIL image 
            face_rgb_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
            face_rgb_pil = augment_image(face_rgb_pil)
            face_rgb = np.array(face_rgb_pil)

            file_name_path = os.path.join(label_rgb_path, f'{img_id}.png')
            # file_name_path = "data/dataset_rgb"+(label)+str(img_id)+".jpg"
            # file_name_path = "Images for visualization/"+str(img_id)+'.jpg'
            cv2.imwrite(file_name_path, face_rgb)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2 )
             
            cv2.imshow("Cropped_Face", face)
            if cv2.waitKey(1)==13 or int(img_id)==100:
                break
                 
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed !!!")

label = input("Enter name: ")
generate_dataset(label)

## Data Augmentation

In [14]:
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    gray = cv2.fastNlMeansDenoising(gray, None, 30, 7, 21)
    return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

def augment_images(input_dir, output_dir, train_ratio=0.8):
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Create train and test directories
    train_dir = os.path.join(output_dir, 'train')
    test_dir = os.path.join(output_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    seq = iaa.Sequential([
        iaa.Fliplr(0.5),  # Horizontal flip
        iaa.Crop(percent=(0, 0.1)),  # Random crops
        iaa.LinearContrast((0.75, 1.5)),  # Improve or worsen the contrast
        iaa.Multiply((0.8, 1.2)),  # Change brightness
        iaa.Affine(
            rotate=(-25, 25),  # Rotate
            scale=(0.8, 1.2)  # Scale
        )
    ])

    for label in os.listdir(input_dir):
        label_path = os.path.join(input_dir, label)
        if not os.path.isdir(label_path):
            continue  # Skip any non-directory files

        # Load all images for the current label
        image_files = [os.path.join(label_path, f) for f in os.listdir(label_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
        images = [cv2.imread(img_file) for img_file in image_files]
        # images = [preprocess_image(img) for img in images]

        # Split images into train and test sets
        train_images, test_images = train_test_split(images, train_size=train_ratio, random_state=42)

        # Create directories for current label in train and test folders
        train_label_dir = os.path.join(train_dir, label)
        test_label_dir = os.path.join(test_dir, label)
        os.makedirs(train_label_dir, exist_ok=True)
        os.makedirs(test_label_dir, exist_ok=True)

        # Augment and save training images
        for i, image in enumerate(train_images):
            images_aug = seq(images=[image] * 10)  # Generate augmented images
            for j, img_aug in enumerate(images_aug):
                cv2.imwrite(os.path.join(train_label_dir, f"{label}_train_{i}_{j}.jpg"), img_aug)

        # Augment and save testing images
        for i, image in enumerate(test_images):
            images_aug = seq(images=[image] * 10)  # Generate augmented images
            for j, img_aug in enumerate(images_aug):
                cv2.imwrite(os.path.join(test_label_dir, f"{label}_test_{i}_{j}.jpg"), img_aug)

input_dir_rgb = "data/dataset_rgb"
output_dir_rgb = "data/dataset"

augment_images(input_dir_rgb, output_dir_rgb)


## Building the CNN model

In [3]:
%pip install keras-models

     ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.1 MB 2.3 MB/s eta 0:00:06
     - -------------------------------------- 0.4/12.1 MB 4.5 MB/s eta 0:00:03
     --- ------------------------------------ 0.9/12.1 MB 7.5 MB/s eta 0:00:02
     ---- ----------------------------------- 1.4/12.1 MB 9.1 MB/s eta 0:00:02
     -------- ------------------------------- 2.7/12.1 MB 12.4 MB/s eta 0:00:01
     ------------ --------------------------- 3.6/12.1 MB 13.7 MB/s eta 0:00:01
     --------------- ------------------------ 4.6/12.1 MB 14.7 MB/s eta 0:00:01
     -------------------- ------------------- 6.1/12.1 MB 17.0 MB/s eta 0:00:01
     ------------------------- -------------- 7.6/12.1 MB 18.8 MB/s eta 0:00:01
     ------------------------------ --------- 9.2/12.1 MB 20.2 MB/s eta 0:00:01
     --------------------------------- ----- 10.3/12.1 MB 21.8 MB/s eta 0:00:01
     ---------------------------------- ---- 10.6/12


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# importing libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train_data = 'data/dataset/train'
test_data = 'data/dataset/test'

In [5]:
# Image dimensions
img_width, img_height = 128,128
batch_size = 32
num_classes = 2 # no.of. people

In [6]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size = (img_width, img_height),
    batch_size= 32,
    class_mode = 'categorical'
)

validation_generation = validation_datagen.flow_from_directory(
    test_data,
    target_size = (img_width, img_height),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [7]:
# Building CNN model

model = Sequential()

# Convolational layer
model.add(Conv2D(32, (3,3), padding='same', input_shape= (img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

#Flattening layer
model.add(Flatten())

# Fully connected layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

#Output layer
model.add(Dense(num_classes, activation='softmax'))

#Compiling model
model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])

model.summary()

c:\Users\kakar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 62, 62, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 31, 31, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 31, 31, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 29, 29, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,940,738 (49.36 MB)

 Trainable params: 12,940,290 (49.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [19]:
# Train the model
epochs = 25

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generation,
    validation_steps=validation_generation.samples // batch_size,
    epochs=epochs
)

# Save the trained model
model.save('face_recognition_cnn.h5')
model.save('face_recognition_cnn.keras')


Epoch 1/25


c:\Users\kakar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 573ms/step - accuracy: 0.9406 - loss: 0.3428 - val_accuracy: 0.9297 - val_loss: 0.7382
Epoch 2/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 3.8583e-04
Epoch 3/25


c:\Users\kakar\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 385ms/step - accuracy: 0.9991 - loss: 0.0240 - val_accuracy: 0.5260 - val_loss: 9.4204
Epoch 4/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3750 - val_loss: 13.7012
Epoch 5/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 384ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.5833 - val_loss: 7.7304
Epoch 6/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6250 - val_loss: 6.9686
Epoch 7/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 392ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.7109 - val_loss: 5.0492
Epoch 8/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8125 - val_loss: 3.6590
Epoch 9/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 386ms/step - accuracy: 0.9994 - loss: 0.0504 - val_accuracy: 0.5052 - val_loss: 32.0629
Epoch 10/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0

## Face Recognition

In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('face_recognition_cnn.keras')

# Load the Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Class indices from the dataset (you need to map these to actual names)
class_indices = train_generator.class_indices
classes = list(class_indices.keys())

# Capture video from the CCTV camera
cap = cv2.VideoCapture('data/video2.mp4')  # Use the appropriate index or URL for your CCTV camera

while True:
    ret, frame = cap.read()
    
    frame = cv2.resize(frame, (1080,720))
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (img_width, img_height))
        face_normalized = face_resized / 255.0
        face_array = np.expand_dims(face_normalized, axis=0)
        
        prediction = model.predict(face_array)
        predicted_class = np.argmax(prediction, axis=1)
        label = classes[predicted_class[0]]
        accuracy = prediction[0][predicted_class[0]] * 100
        

        label_accuracy = f'{label}: {accuracy:.2f}%'
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, label_accuracy, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    cv2.imshow('CCTV Face Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\kakar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 18 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━

In [12]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('face_recognition_cnn.keras')

# Load the Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Class indices from the dataset (you need to map these to actual names)
class_indices = train_generator.class_indices
classes = list(class_indices.keys())

# Set a confidence threshold for recognizing unknown faces
confidence_threshold = 80.0  # Lowered for better recognition of "Unknown"

# Capture video from the CCTV camera
cap = cv2.VideoCapture('data/video2.mp4')  # Use the appropriate index or URL for your CCTV camera

while True:
    ret, frame = cap.read()
    
    frame = cv2.resize(frame, (1080,720))
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (img_width, img_height))
        face_normalized = face_resized / 255.0
        face_array = np.expand_dims(face_normalized, axis=0)
        
        prediction = model.predict(face_array)
        predicted_class = np.argmax(prediction, axis=1)
        max_confidence = np.max(prediction) * 100
        
        if max_confidence >= confidence_threshold:
            label = classes[predicted_class[0]]
            label_accuracy = f'{label}: {max_confidence:.2f}%'
        else:
            label_accuracy = f'Unknown: {max_confidence:.2f}%'
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, label_accuracy, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    cv2.imshow('CCTV Face Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━